In [2]:
## Importing packages needed for scrape
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import requests
import warnings

## Function designed to scrape all of the injuries reported in NBA history within a specific range
def single():
    num = 27700  # Start scraping from this page
    end = 31100  # Stop scraping at this page
    warnings.filterwarnings('ignore', message='The frame.append method is deprecated')
    
    df_list = []  # List to hold DataFrames from each page
    
    while num <= end:
        url = f'https://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=&EndDate=&InjuriesChkBx=yes&start={num}'
        response = requests.get(url)
        html = response.content
        soup = BeautifulSoup(html, 'lxml')
        table = soup.find('table', attrs={'class': 'datatable center'})
        
        # Read the table and add to the list
        temp_df = pd.read_html(str(table))[0]
        df_list.append(temp_df)
        
        num += 25  # Increment by 25 to move to the next page
        time.sleep(2.38)  # Rate limit to avoid overloading the server

    # Combine all DataFrames in the list
    df = pd.concat(df_list, ignore_index=True)

    # Clean and format the final DataFrame
    df.columns = ['Date', 'Team', 'Acquired', 'Relinquished', 'Notes']
    df = df[df['Date'] != 'Date']  # Filter out duplicate headers
    df['Date'] = pd.to_datetime(df['Date'].str.strip(), format='%Y-%m-%d')
    df.reset_index(drop=True, inplace=True)
    df.Acquired = df.Acquired.str.replace('•', '', regex=False)  # Remove • in Acquired
    df.Relinquished = df.Relinquished.str.replace('•', '', regex=False)  # Remove • in Relinquished
    
    return df



In [3]:
df = single()

/var/folders/08/spdh4vf12fsgwsc2hk3q9_c80000gn/T/ipykernel_13047/3063374760.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(table))[0]
/var/folders/08/spdh4vf12fsgwsc2hk3q9_c80000gn/T/ipykernel_13047/3063374760.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(table))[0]
/var/folders/08/spdh4vf12fsgwsc2hk3q9_c80000gn/T/ipykernel_13047/3063374760.py:25: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  temp_df = pd.read_html(str(table))[0]
/var/folders/08/spdh4vf12fsgwsc2hk3q9_c80000gn/T/ipykernel_13047/3063374760.py:25: FutureWarning: Passing literal html to 'r

In [4]:
df.to_csv('NBA Player Injury Stats(2019 - 2024).csv', index = True)